In [156]:
import numpy as np
import pandas as pd
from functools import reduce
import pypfopt

In [7]:
stocks = pd.read_csv("data/sp500_close_data.csv").iloc[:500]

#### Calculation of Historical Returns

In [32]:
ret_list = []
for i in stocks.columns[1:]:
    temp = []
    for j in range(1, len(stocks[i])):
        temp.append((stocks[i][j]/stocks[i][j-1]) - 1)
    ret_list.append(temp)

returns = pd.DataFrame(ret_list).T
returns.columns = stocks.columns[1:]
returns

,AAPL,ABBV,ABT,ACN,ADBE,ADI,ADP,AMAT,AMD,AMGN,...,TXN,UNH,UNP,UPS,V,VRTX,VZ,WFC,WMT,XOM
0,-0.006872,-0.003313,-0.005249,-0.002210,0.001549,-0.000191,-0.003765,0.016345,-0.007500,0.003966,...,0.000213,-0.002009,-0.000502,0.001925,-0.006098,0.005120,0.001801,0.006105,-0.000130,-0.005869
1,0.014141,0.004063,0.000503,0.007383,-0.008509,0.001910,-0.008440,0.043860,-0.007557,0.005754,...,0.000000,0.005159,-0.008937,-0.006245,-0.010303,0.004544,-0.015185,0.000000,-0.000651,0.004503
2,0.011419,0.003862,-0.001758,0.019299,0.001248,0.000381,0.001525,0.006536,0.025381,0.017333,...,-0.002979,0.007887,0.002229,-0.004447,0.000520,-0.004386,-0.002840,-0.000979,0.005475,-0.003486
3,0.003924,0.013192,0.010063,0.001558,0.020259,0.001524,-0.001649,0.010204,0.009901,-0.011834,...,0.011740,-0.008819,0.012688,0.006313,0.003737,0.005782,0.002645,0.011560,0.002463,0.005098
4,-0.002750,-0.003617,-0.002491,-0.000598,0.021995,0.002283,0.011563,0.001837,-0.004902,0.000679,...,0.002110,0.001629,0.007587,-0.000290,0.003675,0.005886,0.002841,0.006779,-0.001423,0.010443
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
494,-0.004154,-0.035228,-0.014118,-0.007939,-0.013370,-0.001082,-0.015571,-0.010579,0.000000,-0.020617,...,-0.010169,-0.008267,0.002073,-0.005509,-0.012982,-0.017409,-0.006441,-0.013466,-0.013935,-0.000447
495,0.011445,0.003485,0.001061,0.002581,0.008027,0.008127,0.002929,0.014257,0.013193,0.004796,...,0.010099,0.007332,0.003528,0.002077,0.008334,0.012390,-0.010020,0.021630,-0.029954,-0.002011
496,-0.003807,-0.019517,-0.013510,-0.004205,-0.008480,-0.002150,-0.005374,-0.000502,-0.018229,-0.014321,...,-0.013446,-0.003908,-0.012609,-0.000099,-0.007103,-0.008567,-0.015082,-0.005550,0.095804,0.008506
497,0.010828,0.006916,0.009667,0.003792,0.010430,0.015081,0.003876,0.138122,0.026525,0.010157,...,0.022715,0.007231,0.011911,0.002172,0.010276,0.045550,0.000604,0.007648,0.009538,-0.004106


#### Define Chromosome (1D Array): Set of genes i.e. fractions of total capital assigned to each stock. Set of weights.

In [4]:
def chromosome(n):
    ''' Generates set of random numbers whose sum is equal to 1
        Input: Number of stocks.
        Output: Array of random numbers'''
    ch = np.random.rand(n)
    return ch/sum(ch)

In [5]:
child=chromosome(100)
print(child,sum(child))

[9.81105085e-03 1.61821184e-02 8.37797407e-03 3.13623104e-03
 9.49301332e-03 3.05208939e-03 1.35149490e-02 6.79388539e-03
 9.14888402e-03 7.68240598e-04 7.27967366e-03 3.06022637e-03
 8.69579377e-03 1.53566052e-02 1.93088785e-02 1.60236312e-02
 7.51748797e-03 3.40894246e-03 9.24103198e-04 9.42789300e-03
 1.18922369e-02 1.60755256e-02 1.00096725e-02 4.77223626e-03
 1.38374030e-02 1.46112763e-02 1.75868053e-02 1.92152908e-02
 1.78982513e-02 6.22225322e-03 9.68757618e-03 1.02929064e-02
 1.64075131e-02 4.55202352e-03 1.90315262e-02 1.55648135e-03
 7.22603238e-04 4.57673356e-03 7.98058967e-03 1.89727129e-02
 1.86046140e-02 1.77154631e-02 8.11441420e-03 1.53590544e-02
 1.16150959e-03 7.87286141e-03 7.12707667e-03 1.58927249e-02
 1.12529073e-02 1.70651081e-02 1.14328245e-02 2.15306274e-03
 1.33086922e-02 5.27499299e-03 7.53618447e-03 5.97610106e-03
 1.85613593e-02 2.82200806e-03 1.78985790e-02 1.07012152e-02
 1.00317199e-02 1.29975798e-02 1.76576382e-02 3.61023599e-03
 4.52169753e-03 1.893203

#### Generate Initial Population (2D Array): A set of randomly generated chromosomes.

In [6]:
n=100 # Number of stocks = 100
pop_size=100 # Initial Population = 100

population = np.array([chromosome(n) for _ in range(pop_size)])
print(population.shape)
print(population)

(100, 100)
[[0.00606112 0.013184   0.00883837 ... 0.00894723 0.01380883 0.0063085 ]
 [0.01222192 0.01689208 0.00591774 ... 0.0136131  0.00305918 0.01383602]
 [0.00407418 0.01411128 0.01784938 ... 0.01285493 0.01135009 0.01537784]
 ...
 [0.00253352 0.00702676 0.00223512 ... 0.01252709 0.00237827 0.00113522]
 [0.00863048 0.01221551 0.01856626 ... 0.01810886 0.0073291  0.00227068]
 [0.01979264 0.00198187 0.00601169 ... 0.00223369 0.01741209 0.01425913]]


#### Calculate Mean, Standard deviation and covariance of the Historical stock returns.

In [33]:
mean_hist_return=returns.mean()
mean_hist_return

AAPL    0.000255
ABBV    0.000355
ABT    -0.000041
ACN     0.000788
ADBE    0.000938
          ...   
VRTX    0.000772
VZ      0.000015
WFC    -0.000007
WMT    -0.000118
XOM    -0.000140
Length: 100, dtype: float64

In [34]:
sd_hist_return=returns.std()
sd_hist_return


AAPL    0.015904
ABBV    0.018930
ABT     0.013798
ACN     0.012400
ADBE    0.016067
          ...   
VRTX    0.031447
VZ      0.009825
WFC     0.012420
WMT     0.012714
XOM     0.013502
Length: 100, dtype: float64

In [35]:
cov_hist_return=returns.cov()
for i in range(100):
    cov_hist_return.iloc[i][i]=0
    
cov_hist_return

,AAPL,ABBV,ABT,ACN,ADBE,ADI,ADP,AMAT,AMD,AMGN,...,TXN,UNH,UNP,UPS,V,VRTX,VZ,WFC,WMT,XOM
AAPL,0.000000,0.000084,0.000092,0.000094,0.000102,0.000134,0.000088,0.000115,0.000102,0.000107,...,0.000118,0.000093,0.000099,0.000067,0.000095,0.000119,0.000055,0.000100,0.000055,0.000083
ABBV,0.000084,0.000000,0.000118,0.000069,0.000105,0.000106,0.000082,0.000114,0.000103,0.000176,...,0.000080,0.000107,0.000088,0.000072,0.000083,0.000199,0.000048,0.000082,0.000041,0.000080
ABT,0.000092,0.000118,0.000000,0.000096,0.000107,0.000102,0.000093,0.000115,0.000119,0.000142,...,0.000098,0.000107,0.000097,0.000078,0.000111,0.000146,0.000062,0.000104,0.000061,0.000078
ACN,0.000094,0.000069,0.000096,0.000000,0.000105,0.000103,0.000094,0.000102,0.000108,0.000111,...,0.000106,0.000087,0.000084,0.000067,0.000101,0.000109,0.000063,0.000096,0.000065,0.000080
ADBE,0.000102,0.000105,0.000107,0.000105,0.000000,0.000123,0.000098,0.000120,0.000105,0.000123,...,0.000109,0.000104,0.000093,0.000070,0.000120,0.000145,0.000058,0.000103,0.000050,0.000069
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
VRTX,0.000119,0.000199,0.000146,0.000109,0.000145,0.000160,0.000119,0.000178,0.000167,0.000260,...,0.000130,0.000174,0.000132,0.000088,0.000125,0.000000,0.000055,0.000131,0.000064,0.000105
VZ,0.000055,0.000048,0.000062,0.000063,0.000058,0.000067,0.000062,0.000064,0.000085,0.000057,...,0.000066,0.000045,0.000049,0.000052,0.000059,0.000055,0.000000,0.000067,0.000047,0.000063
WFC,0.000100,0.000082,0.000104,0.000096,0.000103,0.000115,0.000095,0.000111,0.000122,0.000115,...,0.000108,0.000093,0.000108,0.000080,0.000109,0.000131,0.000067,0.000000,0.000054,0.000092
WMT,0.000055,0.000041,0.000061,0.000065,0.000050,0.000036,0.000057,0.000055,0.000044,0.000060,...,0.000047,0.000056,0.000050,0.000050,0.000056,0.000064,0.000047,0.000054,0.000000,0.000046


#### Calculate Expected returns of portfolio.


In [38]:
def mean_portfolio_return(child):
    return np.sum(np.multiply(child,mean_hist_return))

mean_portfolio_return(population[0])

0.00039932354976542447

#### Calculate portfolio variance.


In [40]:
def var_portfolio_return(child):
    part_1 = np.sum(np.multiply(child,sd_hist_return)**2)
    temp_lst=[]
    for i in range(100):
        for j in range(100):
            temp=cov_hist_return.iloc[i][j] * child[i] * child[j]
            temp_lst.append(temp)
    part_2=np.sum(temp_lst)
    return part_1+part_2

var_portfolio_return(population[0])

9.47963907391055e-05

#### Risk free factor.

In [41]:
rf= 0.0697

#### Fitness Function of a portfolio.


In [110]:
def fitness_fuction(child):
    ''' This will return the Sharpe ratio for a particular portfolio.
        Input: A child/chromosome (1D Array)
        Output: Sharpe Ratio value (Scalar)'''
    return (mean_portfolio_return(child)- ((1+rf)**(1/252)) + 1)/np.sqrt(var_portfolio_return(child))

In [111]:
fitness_fuction(population[10])

0.012419027282875555

#### Select Elite Population (Define a Function): It filters the elite chromosomes which have highest returns, which were calculated in fitness function.

In [112]:
def Select_elite_population(population, frac=0.3):
    ''' Select elite population from the total population based on fitness function values.
        Input: Population and fraction of population to be considered as elite.
        Output: Elite population.'''
    
    fitness_values = np.array([fitness_fuction(individual) for individual in population])
    percentage_elite_idx = int(np.floor(len(population) * frac))
    elite_indices = np.argsort(fitness_values)[-percentage_elite_idx:]
    elite_population = [population[i] for i in elite_indices]
    
    return elite_population


In [69]:
print(len(Select_elite_population(population, frac=0.3)))
Select_elite_population(population, frac=0.3)

30


[array([1.83397299e-02, 1.47554719e-02, 1.07528911e-02, 1.13026261e-02,
        1.74698230e-02, 1.81447776e-02, 1.89599008e-02, 1.79846242e-02,
        1.39406436e-02, 3.57945172e-03, 1.06260566e-02, 1.34038171e-02,
        8.68516083e-03, 2.52851236e-03, 1.55537591e-02, 1.61771519e-02,
        3.52889004e-04, 4.16581998e-03, 1.70822774e-02, 1.91125337e-02,
        5.68666348e-03, 1.12171500e-02, 7.50023056e-03, 1.33000421e-02,
        1.00618382e-02, 1.15762176e-02, 3.30544085e-05, 7.63177807e-03,
        5.35596071e-03, 1.91330537e-02, 7.90516135e-03, 1.06722932e-02,
        9.01366180e-03, 1.05707341e-02, 1.35841131e-02, 1.86970856e-02,
        1.58050033e-02, 1.11363494e-02, 8.13314185e-03, 1.10558200e-02,
        3.85315420e-03, 6.42468218e-03, 1.61178822e-02, 9.60739718e-03,
        1.26539296e-03, 1.45923711e-03, 1.58208625e-02, 8.05481348e-04,
        1.68284659e-02, 3.35660767e-03, 4.57078229e-03, 1.47735764e-02,
        6.87234418e-04, 1.44504645e-02, 4.33246383e-03, 1.134919

#### Mutation: A function that will perform mutation in a chromosome. Randomly choose 2 numbers between [0, 99] and those elements should be swapped.

In [113]:
def mutation(parent, num_exchanges=5):
    ''' 
    Randomly chosen elements of a chromosome are swapped.
    Input: 
        parent: 1D array representing the parent chromosome
        num_exchanges: Number of exchanges to perform
    Output: 
        child: 1D array representing the mutated offspring 
    '''
    child = parent.copy()
    gene_indices = np.arange(len(parent))
    
    for _ in range(num_exchanges):
        n = np.random.choice(gene_indices, 2, replace=False)
        child[n[0]], child[n[1]] = child[n[1]], child[n[0]]
        
    return child

In [50]:
mutation(population[1]),population[1]

(array([0.01222192, 0.01689208, 0.00591774, 0.01601313, 0.00082542,
        0.00704732, 0.01310164, 0.00632287, 0.00564555, 0.01400199,
        0.01569108, 0.01649347, 0.01432378, 0.00810966, 0.01911294,
        0.0004003 , 0.00635523, 0.01221974, 0.0095219 , 0.01748269,
        0.00924853, 0.01548651, 0.01449629, 0.01913424, 0.00268812,
        0.00166716, 0.00850524, 0.00198239, 0.01557363, 0.00960381,
        0.01606589, 0.0158744 , 0.00398983, 0.01104699, 0.00059022,
        0.00372432, 0.00256129, 0.00583493, 0.00846466, 0.00825747,
        0.016635  , 0.00623701, 0.00772865, 0.01289515, 0.01086607,
        0.00168838, 0.01636313, 0.01644574, 0.00707046, 0.0052495 ,
        0.00816244, 0.00930032, 0.01570223, 0.01147224, 0.01070316,
        0.00813021, 0.00645532, 0.00156983, 0.01310679, 0.01411405,
        0.01289877, 0.00796137, 0.01022248, 0.0167177 , 0.00022738,
        0.00237397, 0.01652593, 0.00477667, 0.0065842 , 0.01151689,
        0.0169916 , 0.01554821, 0.01458157, 0.01

#### Crossover: Heuristic crossover or Blend Crossover uses the ﬁtness values of two parent chromosomes to ascertain the direction of the search. It moves from worst parent to best parent.

The oﬀspring are created according to the equation: Off_spring A = Best Parent + β ∗ ( Best Parent − Worst Parent) Off_spring B = Worst Parent - β ∗ ( Best Parent − Worst Parent) Where β is a random number between 0 and 1. This crossover type is good for real-valued genomes

In [114]:
def Heuristic_crossover(parent1,parent2):
    ''' The oﬀsprings are created according to the equation:
            Off_spring A = Best Parent  + β ∗ ( Best Parent − Worst Parent)
            Off_spring B = Worst Parent - β ∗ ( Best Parent − Worst Parent)
                Where β is a random number between 0 and 1.
        Input: 2 Parents
        Output: 2 Children (1d Array)'''
    ff1=fitness_fuction(parent1)
    ff2=fitness_fuction(parent2)

    diff=parent1 - parent2
    beta=np.random.rand()

    if ff1>ff2:
        child1=parent1 + beta * diff
        child2=parent2 - beta * diff
    else:
        child2=parent1 + beta * diff
        child1=parent2 - beta * diff
        
    return child1,child2

In [115]:
def Arithmetic_crossover(parent1,parent2):
    ''' The oﬀsprings are created according to the equation:
            Off spring A = α ∗ Parent1 + (1 −α) ∗ Parent2
            Off spring B = (1 −α) ∗ Parent1 + α ∗ Parent2
            
                Where α is a random number between 0 and 1.
        Input: 2 Parents
        Output: 2 Children (1d Array)'''
    alpha = np.random.rand()
    child1 = alpha * parent1 + (1-alpha) * parent2
    child2 = (1-alpha) * parent1 + alpha * parent2
    return child1,child2

In [58]:
Heuristic_crossover(population[2],population[3])


(array([ 0.00482245,  0.01082614,  0.01151112,  0.01111626,  0.01527615,
        -0.00766888,  0.01540962,  0.01695754,  0.02361508,  0.02238242,
        -0.00224797,  0.00111089,  0.03128303,  0.0302266 ,  0.01675927,
         0.01235319,  0.02142656,  0.01167324,  0.01118766, -0.00061243,
        -0.00284771,  0.00015942,  0.02030076, -0.00231999, -0.01007615,
        -0.01216751,  0.02784526,  0.02386367,  0.00785876,  0.00996362,
         0.02714003,  0.00484771, -0.01159663, -0.00902675,  0.01184536,
         0.01106699,  0.01082151,  0.01038959,  0.00439471,  0.00860147,
         0.00951318,  0.02806223,  0.01409199,  0.02440187,  0.02126053,
         0.01887875,  0.00024139,  0.00881172, -0.0022077 ,  0.00021604,
         0.01318147,  0.00602324, -0.00666315,  0.03352515,  0.01482867,
         0.01283347,  0.01475062, -0.00678974, -0.00210683,  0.00801672,
         0.00666507,  0.00305458,  0.02836391,  0.00163634,  0.00760461,
         0.00138655,  0.02588014,  0.00867018,  0.0

In [55]:
Arithmetic_crossover(population[2],population[3])

(array([0.00442601, 0.01256662, 0.01486915, 0.00770652, 0.0112075 ,
        0.00483763, 0.01208182, 0.00801079, 0.01476438, 0.01978365,
        0.00058061, 0.00513337, 0.01508506, 0.0146591 , 0.01629723,
        0.01531574, 0.01056383, 0.01310581, 0.01215313, 0.00268099,
        0.00297474, 0.01078525, 0.01460307, 0.0037802 , 0.00310376,
        0.00451595, 0.01618771, 0.01139032, 0.0037978 , 0.01479475,
        0.01440622, 0.0031712 , 0.00346766, 0.00245182, 0.01005656,
        0.01175709, 0.01017789, 0.01439934, 0.00267352, 0.01132285,
        0.00668549, 0.01585253, 0.01394479, 0.01851408, 0.01630784,
        0.01415092, 0.00098576, 0.01336003, 0.00662298, 0.00614024,
        0.0103073 , 0.0124852 , 0.00515891, 0.01727321, 0.01446014,
        0.01275235, 0.01545685, 0.00327757, 0.00460414, 0.00639973,
        0.0118914 , 0.00164879, 0.0167646 , 0.00963562, 0.01093799,
        0.00875357, 0.01879362, 0.00999532, 0.00847978, 0.0186876 ,
        0.00635772, 0.01375116, 0.01079544, 0.01

#### Next Generation: A function which does mutation, mating or crossover based on a probability and builds a new generation of chromosomes.

In [116]:
def next_generation(pop_size,elite,crossover=Heuristic_crossover):
    ''' Generates new population from elite population with mutation probability as 0.4 and crossover as 0.6. 
        Over the final stages, mutation probability is decreased to 0.1.
        Input: Population Size and elite population.
        Output: Next generation population (2D Array).'''
    
    new_population=[]
    elite_range=range(len(elite))

    while len(new_population) < pop_size:
        if len(new_population) > 2*pop_size/3: # In the final stages mutation frequency is decreased.
            mutate_or_crossover = np.random.choice([0, 1], p=[0.9, 0.1])
        else:
            mutate_or_crossover = np.random.choice([0, 1], p=[0.4, 0.6])
        
        if mutate_or_crossover:
            indx = np.random.choice(elite_range)
            new_population.append(mutation(elite[indx]))

        else:
            p1_idx,p2_idx = np.random.choice(elite_range,2)
            c1,c2 = crossover(elite[p1_idx],elite[p2_idx])
            chk = 0
            for gene in range(100):
                if c1[gene] < 0:
                    chk += 1
                else:
                    chk += 0

            if chk > 0:
                p1_idx,p2_idx = np.random.choice(elite_range,2)
                c1,c2 = crossover(elite[p1_idx],elite[p2_idx])

            new_population.extend([c1,c2])
    return new_population

In [75]:
elite = Select_elite_population(population)
next_generation(100,elite)[:3]

[array([-0.00130779,  0.0158995 ,  0.01071903, -0.00134771,  0.0190462 ,
         0.0192795 ,  0.02602267,  0.00508837,  0.02433383,  0.00292602,
        -0.00516288, -0.00065652,  0.0014692 ,  0.01895577, -0.0030546 ,
         0.00738048,  0.01039641,  0.02043744,  0.00667545,  0.01249665,
         0.02038444,  0.0108542 ,  0.02585561,  0.00829457,  0.01910429,
         0.0021654 ,  0.00782107,  0.02169318, -0.00072883, -0.00010642,
         0.02409843,  0.01955239, -0.00983215, -0.00228355,  0.01768109,
        -0.00883497,  0.032865  ,  0.02000452,  0.01979777,  0.01603364,
        -0.01246246,  0.00555259, -0.00195965,  0.01425136, -0.00881275,
         0.00842178,  0.0116834 ,  0.02146832,  0.02000652,  0.01823349,
         0.02900517,  0.02586416,  0.0046469 ,  0.00217938,  0.025036  ,
         0.00627036, -0.00104404,  0.02531469,  0.01200225, -0.00247216,
         0.01057775, -0.00627632, -0.00864737,  0.02590697, -0.00859317,
         0.01442264,  0.02077701,  0.00339305,  0.0

In [76]:
next_generation(100,elite,Arithmetic_crossover)[:3]

[array([0.01694297, 0.01542148, 0.0143067 , 0.01590883, 0.01705424,
        0.00854231, 0.01130928, 0.00128119, 0.01416343, 0.0133654 ,
        0.00348759, 0.01760255, 0.01207212, 0.00113409, 0.00169135,
        0.01821148, 0.01177481, 0.00988732, 0.00872946, 0.01696941,
        0.0149788 , 0.0002171 , 0.00850552, 0.00918791, 0.010912  ,
        0.00781384, 0.01741199, 0.00940062, 0.01199529, 0.00811781,
        0.01858572, 0.00049376, 0.01547245, 0.01675458, 0.0149676 ,
        0.00764204, 0.01638827, 0.00326641, 0.00885889, 0.00696083,
        0.00259089, 0.00729774, 0.01254212, 0.01696428, 0.01739636,
        0.00494319, 0.00978201, 0.01529817, 0.00086413, 0.00815276,
        0.0009449 , 0.00578832, 0.01548512, 0.00443262, 0.01717462,
        0.00048199, 0.01031867, 0.01016552, 0.01234952, 0.01769289,
        0.01131543, 0.00318716, 0.01114612, 0.01167934, 0.00968871,
        0.01073714, 0.0087983 , 0.00743021, 0.00950639, 0.00705808,
        0.0045888 , 0.00496561, 0.01489983, 0.01

#### Iterate the process: Iterate the whole process till their is no change in maximum returns/min risk or for fixed number of iterations.

In [119]:
n=100 # Number of stocks = 100
pop_size=100 # initial population = 100

# Initial population
population = np.array([chromosome(n) for _ in range(pop_size)])

# Get initial elite population
elite = Select_elite_population(population)

iteration=0 
Expected_returns=0
Expected_risk=1

while iteration <= 40:
    print('Iteration:',iteration)
    population = next_generation(100,elite,Arithmetic_crossover)
    elite = Select_elite_population(population)
    Expected_returns=mean_portfolio_return(elite[0])
    Expected_risk=var_portfolio_return(elite[0])
    print('Expected returns of {}% with risk of {}% and Sharpe Ratio of {}\n'.format((((1+Expected_returns)**252)-1)*100 , (Expected_risk**0.5)*np.sqrt(252)*100 , ((((1+Expected_returns)**252)-1)*100 - (rf*100))/ ((Expected_risk**0.5) * np.sqrt(252) *100)))
    iteration+=1

Iteration: 0
Expected returns of 12.171583512446315% with risk of 15.106279027134551% and Sharpe Ratio of 0.3443325456323828

Iteration: 1
Expected returns of 12.869631676690062% with risk of 15.348298814854674% and Sharpe Ratio of 0.3843834256719169

Iteration: 2
Expected returns of 13.069109956641345% with risk of 15.256364550667461% and Sharpe Ratio of 0.3997747914574125

Iteration: 3
Expected returns of 13.223937306504997% with risk of 15.368230054525872% and Sharpe Ratio of 0.40693933421846723

Iteration: 4
Expected returns of 13.326949045039127% with risk of 15.236448451069466% and Sharpe Ratio of 0.4172198701983549

Iteration: 5
Expected returns of 13.362880680855383% with risk of 15.139898678013438% and Sharpe Ratio of 0.4222538615888688

Iteration: 6
Expected returns of 13.451021294482967% with risk of 15.267737421204894% and Sharpe Ratio of 0.4244912730475489

Iteration: 7
Expected returns of 13.577427150787935% with risk of 15.255675324064102% and Sharpe Ratio of 0.433112727

In [120]:
for i in list(range(100)):
    print(returns.columns[i],':',elite[0][i]) 

AAPL : 0.006296047909030498
ABBV : 0.011936926594502933
ABT : 0.009159743753683714
ACN : 0.008757236481997107
ADBE : 0.010091108667392126
ADI : 0.008724345473783108
ADP : 0.008452096623261945
AMAT : 0.008904211614213318
AMD : 0.008435953819053425
AMGN : 0.0091453299699585
AMZN : 0.01720321939929794
AVGO : 0.013164816492202316
AXP : 0.004622768633172317
BA : 0.007295088318042001
BAC : 0.004491955050500844
BKNG : 0.006784048863119982
BLK : 0.006707240298981718
BRK-B : 0.010399001454132244
BSX : 0.01004892601682744
C : 0.008610246858171395
CAT : 0.0062939727114149045
CB : 0.008317444447626207
CI : 0.01278059323975729
CMCSA : 0.011873029922849505
COP : 0.003943211001075759
COST : 0.006190240758750585
CRM : 0.011380449184507814
CSCO : 0.010274302372552484
CVX : 0.0065314103383789235
DE : 0.008108540693163198
DHR : 0.012193175635047949
DIS : 0.00992993778327306
ELV : 0.010734000517584724
ETN : 0.0024621554094353564
FI : 0.016816103935383967
GE : 0.010369230758780438
GOOG : 0.0146169170142200

In [122]:
print('Expected returns of {}% with risk of {}% and Sharpe Ratio of {}\n'.format((((1+Expected_returns)**252)-1)*100 , (Expected_risk**0.5)*np.sqrt(252)*100 , ((((1+Expected_returns)**252)-1)*100 - (rf*100))/ ((Expected_risk**0.5) * np.sqrt(252) *100)))


Expected returns of 15.083142656203542% with risk of 15.233461800224596% and Sharpe Ratio of 0.5325869301805009



In [174]:
def calculate_discrete_allocation(weights, latest_prices, total_portfolio_value):

    weights = list(weights.items())
    weights.sort(key=lambda x: x[1], reverse=True)

    allocation = {}
    leftover_funds = total_portfolio_value
    shares_bought = [0] * len(weights)
    buy_prices = [price for _, price in latest_prices.items()]

    for idx, (ticker, weight) in enumerate(weights):
        price = latest_prices[ticker]
        n_shares = int(weight * total_portfolio_value / price)
        cost = n_shares * price
        assert cost <= leftover_funds, "Unexpectedly insufficient funds."
        leftover_funds -= cost
        shares_bought[idx] = n_shares

    while leftover_funds > 0:

        current_weights = np.array(buy_prices) * np.array(shares_bought)
        current_weights = current_weights.astype(float) 
        current_weights /= current_weights.sum()
        
        ideal_weights = np.array([i[1] for i in weights])
        deficit = ideal_weights - current_weights

        idx = np.argmax(deficit)
        ticker, weight = weights[idx]
        price = latest_prices[ticker]

        counter = 0
        while price > leftover_funds:
            deficit[idx] = 0
            idx = np.argmax(deficit)

            if deficit[idx] < 0 or counter == 10:
                break

            ticker, weight = weights[idx]
            price = latest_prices[ticker]
            counter += 1

        if deficit[idx] <= 0 or counter == 10:
            break

        shares_bought[idx] += 1
        leftover_funds -= price

    allocation = {ticker: shares for (ticker, _), shares in zip(weights, shares_bought) if shares != 0}

    return allocation, leftover_funds


total_portfolio_value = 10000
weights = dict(map(lambda i,j : (i,j) , returns.columns ,elite[0]))
prices = stocks.iloc[-1, 1:]

allocation, leftover = calculate_discrete_allocation(weights, prices, total_portfolio_value)
for stock, num_shares in allocation.items():
    print(f"Allocate {num_shares} shares of {stock}")


Allocate 6 shares of AMZN
Allocate 2 shares of UNH
Allocate 4 shares of FI
Allocate 3 shares of SBUX
Allocate 2 shares of META
Allocate 2 shares of NOW
Allocate 3 shares of KO
Allocate 5 shares of LOW
Allocate 23 shares of GOOG
Allocate 1 shares of REGN
Allocate 3 shares of MMC
Allocate 2 shares of PANW
Allocate 2 shares of MSFT
Allocate 12 shares of NVDA
Allocate 5 shares of PEP
Allocate 1 shares of AVGO
Allocate 1 shares of SPGI
Allocate 2 shares of SNPS
Allocate 3 shares of MDLZ
Allocate 2 shares of CI
Allocate 1 shares of INTU
Allocate 1 shares of NFLX
Allocate 1 shares of V
Allocate 2 shares of DHR
Allocate 8 shares of TSLA
Allocate 1 shares of MDT
Allocate 2 shares of ABBV
Allocate 3 shares of CMCSA
Allocate 1 shares of PG
Allocate 1 shares of GS
Allocate 1 shares of CRM
Allocate 1 shares of JPM
Allocate 1 shares of LLY
Allocate 3 shares of NEE
Allocate 3 shares of GOOGL
Allocate 1 shares of LMT
Allocate 1 shares of SYK
Allocate 1 shares of ELV
Allocate 1 shares of NKE
Allocate 1